In [10]:
from IPython.display import Image

# Gridding

Here we present multiple solutions and approaches on the gridding problem for a potentially huge dataset.

## Summary

- We achieve a **10 fold** increase...
- scaling
- C
- CUDA

In what follows we put the detailed study of the different approaches and benchmarking results for different versions: 

## 1. `python/` 

Contains the treatment of the problem in pure python, the approaches taken on each of the versions are as follow:

   - `v0_original.py`: The original version of the code.
   - `v1_index_calc_jitted.py`: Calculating the grid indices has been Just In Compiled (JIT).
   - `v2_gridding_jitted.py`: The whole gridding function has been compiled.
   - `v3_single_timestep_vectorized.py`: Calculation of a single timestep of the grid has been fully vectorized using numpy intrinsic functions.
   - `v4_single_timestep_vectorized_jitted.py`: On top of vectorizing a single timestep the function for calculating a single timestep of the grid has been compiled.
   - `v5_gridding_vectorized.py`: The whole gridding function has been fully vectorized using numpy.
   - `v6_gridding_vectorized_multithreaded.py`: On top of vectorizing the gridding function it uses python `concurrent` library to parallelize the gridding over `n_workers` threads using chunks of the dataset.
   - `v7_mpi_timesteps.py`: Using `mpi4py` library we divide the computation of the grid over timsteps to multiple processes.
   - `v8_mpi_baselines.py`: Same as above but divide the dataset over baseline pairs wrather than timesteps to multiple process.

### 1.1 Benchmarks

Here we present the benchmarking results obtained on pure python implementations. Note that these benchmarks are done on a single node, with *dual sockets* and an *AMD EPYC 7H12 64-Core Processor* per socket.

#### 1.1.1 Original

<center><img src="python/plots/v0_original.png" alt="v0"/></center>

From the benchmark above we see that the most time consuming and the bottleneck is the `gridding` function as we expected from the 3 nestes loops in python! For this reason we will focus our attention on the gridding function and will present the different results and strategies on that.

Here we put the benchamrking for the different version of the code, without any explicit parallelism by us (e.g. the benchmarks below are for the versions 1 to 5 without any multi-threading or use of MPI multi-processing)

#### 1.1.2 Versions 1 to 5

<center><img src="python/plots/v1tov5.png" alt="v1tov5"/></center>

#### 1.1.3 Version 6 - Multithreaded

<center><img src="python/plots/v6.png" alt="v6"/></center>


#### 1.1.4 Versions 7/8 MPI distributed


<center><img src="python/plots/v7v8.png" alt="v7v8"/></center>

## 2.`C_python/`

Here we offload the gridding function to C and use the `ctypes` library to load the shared library. Different version of the sources are as follow:

- `libgrid.c`: The library cotaining different implementations of the gridding function. To make the library an MPI compiler must be present, then issue `make` in the directory which uses GNU `mpicc` compiler with `-O3` optimization.
- `v1_omp.py`: Uses `gridding_omp` function which parallelized the gridding with OpenMP threads.
- `v2_mpi_omp.py`: Uses the hybrid MPI/OpenMP `grdding_mpi_omp` function, divides the grid in timesteps over MPI processes and parallelizes the loop with OpenMP threads.
- `v3_simd.py`: Uses the `gridding_simd` function which implements a fully vectorized gridding manually, note that with `-O3` flag, compiler basically does that for us, but here we have done it manually also as an exercise.
- `v4_simd_mpi_omp.py`: Uses the `grdding_simd_mpi_omp` function, which basically combines all the previous 3 implementations, divides the grid over timesteps to multiple processes, parallelizes the loop with OpenMP threads over a fully vectorized gridding implementation.

### 2.1 Benchmarks

The benchmarks here are also done on the same EPYC node. Note the significant speedup and scaling we achieve by offloading the gridding function to C.

#### 2.1.1 Version 1 - OpenMP Threaded

| Version 1 (OpenMP): Number of Threads  | Time(s) |
| ------------------ | --------|
| 1   | 0.316 |
| 2   | 0.172 |
| 4   | 0.094 |
| 8   | 0.053 |
| 16  | 0.035 |
| 32  | 0.034 |
| 64  | 0.037 |
| 128 | 0.048 |

<center><img src="C_python/plots/v1.png" alt="v1"/></center>

Note that with the C OpenMP implementation we achieve **4 orders of magnitude** speedup compared to the original version. Note that the best scaling achieved upto 32 threads and after that synchronization between threads overwhelms the gain in threading and as we move toward more threads we acutally spent more time on threading overhead than gaining. Note also that compared to python threads we almost have a speedup of **100 times**.

#### 2.1.2 Version 2 - MPI distributed


| Version 2 (MPI): Number of Processes  | Time(s) |
| ------------------ | --------|
| 1   | 0.340 |
| 2   | 0.198 |
| 4   | 0.181 |
| 8   | 0.170 |
| 16  | 0.198 |
| 32  | 0.306 |
| 64  | 0.386 |
| 128 | 0.674 |

<center><img src="C_python/plots/v2.png" alt="v2"/></center>

Note that with MPI parallelization we achieve a scaling upto 8 processes and after that the **ccommunication time** between processes takes over the computation which is expected for a problem of this size, we actually need a large dataset to see the benifits of MPI distribution over multiple nodes. Note that here most of the time is spent in the `MPI_Reduce` call inside the library. Also an interesting fact is that because of this we almost perform **10 times** worse than OpenMP threads but **10 times** better than MPI launched directly with python.

#### 2.1.3 Version 3 - SIMD/OpenMP


| Version 3 (SIMD/OpenMP): Number of Threads  | Time(s) |
| ------------------ | --------|
| 1   | 0.162 |
| 2   | 0.088 |
| 4   | 0.045 |
| 8   | 0.030 |
| 16  | 0.026 |
| 32  | 0.030 |
| 64  | 0.034 |
| 128 | 0.045 |

<center><img src="C_python/plots/v3.png" alt="v3"/></center>

Note that as we have compiled the code with `-O3` flag compiler can do vectorization as it can deduce. Here we have implemented a manual vectorization using SIMD intrinsics. We see almost **60% speedup** over a pure OpenMP implementation with 16 threads, which is also compiled with `-O3`.

#### 2.1.4 Version 4 - SIMD/MPI/OpenMP


| Version 4 (SIMD/MPI/OpenMP): Number of Processes  | Time(s) |
| ------------------ | --------|
| 1   | 0.162 |
| 2   | 0.088 |
| 4   | 0.045 |
| 8   | 0.030 |
| 16  | 0.026 |
| 32  | 0.030 |
| 64  | 0.034 |
| 128 | 0.045 |

<center><img src="C_python/plots/v3.png" alt="v3"/></center>

## 3.`CUDA_python/`

Here we offload the gridding function to CUDA and use the `ctypes` library to load the shared library. Different version of the sources are as follow:

- `libgrid.cu`: The library cotaining the single GPU implementation of the gridding function. To compile it issue `make` in the directory, for which a CUDA capable compiler is needed, which here we have uses `nvcc` from NVIDIA SDK.
- `libgird_mpi.cu`: The library containing the multi-GPU implementation of the gridding function using CUDA and MPI. To compile it issue `make mpi` in the directory, for which linking against the MPI library is needed, note that here we don't assume a CUDA aware implementation, hence this forces us extra data movement when we are reducing the results from different GPUs.
- `v1_cuda_.py`: Uses `libgrid` shared library to perform gridding on a single GPU.
- `v2_cuda_mpi.py`: Uses `libgrid_mpi` shared library so perform gridding on multiple GPUs. The gridding over timsteps is divided between different GPUs using MPI processes.

### 3.1 Benchmarks

The GPU benchmarks are done a node with 2 NVIDIA V100 SXM2 32GB GPUs with an Intel Xeon Gold 6226 CPU.

#### 3.1.1 Version 1 - Single GPU


| Function  | Time(s) |
| ------------------  | ------|
| `gridding`          | 1.444 |
| `gridding_kernel`   | 0.005 |

<center><img src="CUDA_python/plots/v1.png" alt="v1"/></center>

The actual gridding kernel which is done on GPU is done in the order or milliseconds, most of the time is spent on **data copy** between CPU and GPU, which is the bottleneck.

#### 3.1.2 Version 2 - Multi-GPU


| Version 2 (CUDA/MPI): Number of MPI Processes  | `gridding` Time(s)  | `gridding_kernel`  Time(s) |
| ------------------ | --------| ------- |
| 1   | 1.526 | 0.005 | 
| 2   | 1.608 | 0.003 |

<center><img src="CUDA_python/plots/v2.png" alt="v2"/></center>

We observe that the kernel scales almost lineatly to 2 GPUs, but then the whole gridding function because of data movement between GPUs and CPUs has remained almost the same, which prohibits the further scaling over GPUs. The problem size here for two GPUs is not that large and hence the **communication time and the data movement** part takes over the actual computation pretty quickly and by 3 orders of magnitude